# Recalculating Accretion Rates:

### 1. Accretion luminosity:
    
*Variables needed:*
        
1. name of emission line
    
2. emission line flux
    
3. object distance

*Procedure:*

1. Turn line flux into line luminosity using distance estimate, or existing line luminosity into an updated one using updated distance estimate

2. Turn line luminosity into accretion luminosity using scaling relations from Alcala (2017).

### 2. Accretion rate:

*Variables needed:*

1. accretion luminosity

2. object mass (from spectral type to temperature to evolutionary tracks)

3. object radius (from evolutionary tracks as well)

4. magnetospheric radius (assumed to be 5 \* object radius)

In [1]:
from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun
import pandas as pd

In [2]:
#import scaling relation coefficients
rel = pd.read_csv('scaling_relations_alcala2017_copy.csv')
rel.set_index('Tracer', inplace=True)

In [3]:
#line luminosity functions

def line_lum_distconv(line_lum_old, dist_old, dist_new):
    """
    Calculate the line luminosity at a distance
    given the line luminosity at another distance.
    
    """
    line_lum_new = line_lum_old * (dist_old**2)/(dist_new**2)
    return line_lum_new

def line_lum(line_flux, dist):
    """
    Calculate line luminosity given line flux and distance
    assuming line flux is extinction corrected.
    """
    #future: could add extinction correction in there? Not sure how that works.
    
    line_lum = 4 * pi * dist**2 * line_flux
    return line_lum

In [51]:
def accr_lum(L_line, tracer, L_line_err = 0*u.W):
    """
    Translate a line luminosity to accretion luminosity using empirical
    relationships from Alcala et al. 2017.
    
    Included tracers are:
    'H_alpha'
    'H_beta'
    'H_gamma'
    'H_delta'
    'H_epsilon'
    'Pa_beta'
    'Pa_gamma'
    'He_I_587.6'
    'He_I_667.8'
    'Ca_II_K'
    'Ca_II_H'
    'Ca_II_866.2'
    """
        
    a, a_err, b, b_err = rel['a'][tracer],rel['a_err'][tracer],rel['b'][tracer],rel['b_err'][tracer]
    
    log_L_acc = b + a * log10(L_line/L_sun)
    
    L_acc = 10**log_L_acc*L_sun
    
    
    #attempt at error propagation
    #having problems with units here
    
    c_err = (L_line_err)/(log(10) * L_line)
    ac_err = a * log10(L_line/L_sun) * ((a_err/a)**2 + (c_err/log10(L_line/L_sun))**2)**0.5
    log_L_acc_err = (b_err**2 + ac_err**2)**0.5
    L_acc_err = L_acc * log(10) * log_L_acc_err

    return L_acc/L_sun, L_acc_err/L_sun

***Error propagation:*** *this is how I **think** the error propagation should work for the scaling relation. Need to check this.*

Overall formula: $\log (L_{acc}/L_{\odot}) = a \cdot \log(L_{line}/L_{\odot}) + b$

Errors:

$\delta c = \delta \log(L_{line}/L_{\odot}) = \frac{1}{\ln10} \frac{\delta L_{line}}{L_{line}}$

$\delta ac = ac \cdot \sqrt{(\frac{\delta a}{a})^2 + (\frac{\delta c}{c})^2}$

$\delta L_{acc} = \delta (ac + b)= \sqrt{(\delta ac)^2 + (\delta b)^2}$

In [52]:
accr_lum(10**22 * u.W, 'H_alpha', 10**20 * u.W)

(<Quantity 0.00036412>, <Quantity 0.00024961>)